In [2]:
import pandas as pd
import joblib
import requests

In [4]:
# Charger le modèle et le préprocesseur + prédiction
pricing = pd.read_csv('src/get_around_pricing_project.csv').iloc[:,1:]

list = ['Alfa Romeo', 0, 66, 'diesel', 'beige', 'convertible', 1, 1, 1, 1, 1, 1, 1]

data = pd.DataFrame([list], columns=pricing.drop("rental_price_per_day", axis=1).columns)

loaded_preprocessor = joblib.load('src/preprocessor.pkl')
data = loaded_preprocessor.transform(data)

loaded_model = joblib.load('src/model.pkl')
prediction = loaded_model.predict(data)

prediction

array([140.31218846])

In [5]:
# Prédiction via API

list = ['Alfa Romeo', 0, 66, 'diesel', 'beige', 'convertible', 1, 1, 1, 1, 1, 1, 1]
# api_url = "http://localhost:4000/predict"
api_url = "https://getaround-fastapi1-f159113e9f42.herokuapp.com/predict"
data = {"filters_list": list}
response = requests.post(api_url, json=data)

if response.status_code == 200:
    result = response.json()
    print(result['prediction'])
else:
    print("Erreur lors de la requête à l'API.")

140.31218845857995
